In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# Load the California Housing dataset
california = fetch_california_housing()

In [2]:
california.data.shape

(20640, 8)

In [3]:
california.data[:5]

array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02],
       [ 7.25740000e+00,  5.20000000e+01,  8.28813559e+00,
         1.07344633e+00,  4.96000000e+02,  2.80225989e+00,
         3.78500000e+01, -1.22240000e+02],
       [ 5.64310000e+00,  5.20000000e+01,  5.81735160e+00,
         1.07305936e+00,  5.58000000e+02,  2.54794521e+00,
         3.78500000e+01, -1.22250000e+02],
       [ 3.84620000e+00,  5.20000000e+01,  6.28185328e+00,
         1.08108108e+00,  5.65000000e+02,  2.18146718e+00,
         3.78500000e+01, -1.22250000e+02]])

In [4]:
california.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [5]:
california.target_names

['MedHouseVal']

In [6]:
df = pd.DataFrame(california.data)

In [7]:
df.columns = california.feature_names

In [8]:
df['MedHouseVal'] = california.target 
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [9]:
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.DataFrame(california.target, columns=["Median_House_Value"])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [13]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train.values.ravel())

RandomForestRegressor(random_state=42)

In [14]:
y_train.values.shape

(16512, 1)

In [15]:
y_pred_rf = rf_model.predict(X_test_scaled)

In [16]:
print("Random Forest Regression Metrics:")
print(f"MAE: {mean_absolute_error(y_test, y_pred_rf)}")
print(f"MSE: {mean_squared_error(y_test, y_pred_rf)}")
print(f"R2 Score: {r2_score(y_test, y_pred_rf)}")

Random Forest Regression Metrics:
MAE: 0.3274252027374032
MSE: 0.255169737347244
R2 Score: 0.8052747336256919


In [17]:
with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

In [18]:
import tensorflow as tf
from tensorflow import keras

# Build the model
dl_model = keras.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1)  # Output layer for regression
])

C:\Users\yasha\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
dl_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [20]:
dl_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=00)

Epoch 1/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.8990 - mae: 0.7761 - val_loss: 0.4313 - val_mae: 0.4596
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3922 - mae: 0.4483 - val_loss: 0.3723 - val_mae: 0.4232
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3609 - mae: 0.4306 - val_loss: 0.3460 - val_mae: 0.4121
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3267 - mae: 0.4046 - val_loss: 0.3302 - val_mae: 0.3944
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3285 - mae: 0.4003 - val_loss: 0.3268 - val_mae: 0.4008
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3180 - mae: 0.3933 - val_loss: 0.3181 - val_mae: 0.3871
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3009 - mae: 0.3831 - val_loss: 0.3188 - val_mae: 0.4068
Epoch 8/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3116 - mae: 0.3881 - val_loss: 0.3069 - val_mae: 0.3817
Epoch 9/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [21]:
dl_model.save("deep_learning_model.h5")

In [22]:
with open("random_forest_model.pkl", "rb") as f:
    loaded_rf_model = pickle.load(f)


y_pred_loaded_rf = loaded_rf_model.predict(X_test_scaled)
print(f"Loaded RF Model R2 Score: {r2_score(y_test, y_pred_loaded_rf)}")

Loaded RF Model R2 Score: 0.8052747336256919


In [27]:
from tensorflow.keras.losses import MeanSquaredError  # Or from keras.losses if using an older Keras

loaded_dl_model = keras.models.load_model("deep_learning_model.h5", custom_objects={'mse': MeanSquaredError()})

y_pred_loaded_dl = loaded_dl_model.predict(X_test_scaled)
print(f"Loaded DL Model MSE: {r2_score(y_test, y_pred_loaded_dl)}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Loaded DL Model MSE: 0.7995166778564453
